# **GATEWAY MAIN PROGRAM FOR IOT LAB**
*Author: Minh Nguyen*  
*Date: October 30th, 2021*

## **Overview**
___
The bulk of this python program is to running a simple gateway that is capable of receiving data from the [Adafruit server](https://io.adafruit.com/ "go to Adafruit main page") (broker) and dispatching it to an IoT Arduino node. The gateway also monitors the data sent from the Arduino and publishes it to the server, therefore establishing a form of communication between the broker and the sensor.

The attributes of this simple implementation of gateway are:
- The gateway receieves **temperature** and **humidity** data from the Arduino and publishes them to the broker.
- The gateway listens to **led** and **servo** data from the broker and dispatches them to the Arduino.
- The communication protocol between the gateway and the broker is **MQTT**, which is implemented through the use of the [paho-mqtt](https://pypi.org/project/paho-mqtt/ "go to paho-mqtt pydoc") library.
- The communication protocol between the gateway and the Arduino is **serial UART**.

## **Implementation**
___
I will walk you through the detail implementation of this python program, including the idea and the approach I used to establish a stable communication between the gateway and the broker.

 ### **The keep-in-mind tactic, recommended by Adafruit themselves.**
The Adafruit broker has its own [API](https://pypi.org/project/adafruit-io/) for the end-users. This API is optimized and provides many ease-of-use interfaces. Because we are not going to use the Adafruit API, we need to comply with the best practices recommended by the broker. The quote below is cited from the Adafruit IO MQTT api [documentation](https://io.adafruit.com/api/docs/mqtt.html#adafruit-io-mqtt-api), under the section entitled *Writing Safe MQTT Code*:

> **Always. Use. Delay.**  
> Make sure there is time in between messages to the Adafruit IO MQTT broker. If you're saving data in an Arduino or CircuitPython sketch inside the program's main loop, make sure you're delaying, sleeping, or using some other technique to put space in between messages.

During implementation, I reckoned that there are many routines of the MQTT protocol that run on seperate threads. For example the callback to `on_message()` or `on_disconnect()` function is invoked on its own thread, without any interfering to or impacted from the big `while` of the gateway. This is kinda out of control because according to the safe guide from Adafruit, we must have someway to control the delay between incoming messages. In this situation, because we have no way to tell when the message will come, there are high chances the broker will send out a lot of data to the gateway. If the `on_message()` callback were to handle all data and dispatch to the Arduino, this would lead to serious problem as instructed by the Adafruit documentation. To address this, I use a **shared-buffer queue** for the incoming messages. The `on_message()` callback only bother putting the incoming data onto the queue, and our big `while` loop will constantly check if the queue is empty or not. This way, we can put our control to the `while` loop, simply by inserting a `time.delay()` into it, and the Arduino will happily received data from the dispatching of the while loop without fearing of data lost. The pseudocode is as follow:

    messageBuffer = queue.Queue()

    def on_message(data, ...):
        messageBuffer.put(data)

    ...

    while True:
        if not messageBuffer.empty():
            data = messageBuffer.get()
            dispatch(data)
        
        time.delay(SCAN_DELAY)

### **The lock-thread behavior of routines.**
The out-thread behavior of callbacks not only making the control of incoming messages harder, but also put the data race problems at high risk. There are 2 main concerns regarding this issue:  
- the order of debuggings message on terminal
- the use of `userdata` object

For the first concern, consider this piece of code:

    gtw = mqtt.Client(...)

    ...

    gtw.connect()